In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data_train_avg = pd.read_csv("../Data/train_avg_age.csv")
data_valid_avg = pd.read_csv("../Data/val_avg_age.csv")
data_test_avg = pd.read_csv("../Data/test_avg_age.csv")

data_train_knn = pd.read_csv("../Data/train_knn_age.csv")
data_valid_knn = pd.read_csv("../Data/val_knn_age.csv")
data_test_knn = pd.read_csv("../Data/test_knn_age.csv")

In [3]:
data_train_avg

,affiliate_channel,affiliate_provider,age,country_destination,dac_month,first_affiliate_tracked,first_browser,first_device_type,gender,language,signup_app,signup_flow,signup_method,tfa_month
0,2.0,4.0,37.0,11,7,6.0,43.0,3.0,0.0,6.0,2.0,0.0,0.0,7
1,2.0,4.0,37.0,11,11,0.0,43.0,3.0,0.0,6.0,2.0,0.0,0.0,11
2,7.0,8.0,37.0,4,2,0.0,8.0,3.0,1.0,6.0,2.0,0.0,0.0,2
3,2.0,4.0,27.0,4,4,0.0,8.0,6.0,1.0,6.0,2.0,0.0,1.0,4
4,5.0,1.0,64.0,11,2,3.0,22.0,6.0,2.0,6.0,2.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170755,2.0,4.0,37.0,6,8,6.0,8.0,6.0,0.0,6.0,2.0,0.0,0.0,8
170756,2.0,4.0,26.0,11,12,0.0,43.0,3.0,1.0,6.0,2.0,0.0,1.0,12
170757,2.0,4.0,58.0,4,9,6.0,8.0,6.0,1.0,6.0,2.0,0.0,0.0,9
170758,2.0,4.0,58.0,4,9,0.0,43.0,3.0,1.0,6.0,2.0,0.0,0.0,9


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device.".format(device))

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_chain = nn.Sequential(
            nn.Linear(13, 256),
            nn.ReLU(),
            nn.LayerNorm(256),
            #nn.Linear(512, 512),
            #nn.ReLU(),
            #nn.LayerNorm(512),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.LayerNorm(128),
            nn.Linear(128, 12),
            nn.ReLU()
            #nn.Softmax(dim = -1)
        )
    def forward(self, x):
        result = self.linear_chain(x)
        return result

# model = NeuralNetwork().to(device)
# print(model)

Using cpu device.


In [5]:
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# batch_size = 64

def train(x_train, y_train, model, loss_fn, optimizer):
    size = x_train.shape[0]
    for t in range(size // batch_size):
        X = x_train[t*batch_size:min((t+1)*batch_size, size)]
        y = y_train[t*batch_size:min((t+1)*batch_size, size)]
        
        pred = model(X).type(torch.float)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if t % 200 == 0:
            current = min((t+1)*batch_size, size)
            loss= loss.item()
            print(f"loss: {loss:>7f}, [ current: {current}/{size} ]")

In [6]:
def get_dcg(index):
    return 1 / np.log2(index + 2)

In [7]:
def get_ndcg(y_pred, y_true):
    ngdcg = 0.0
    
    for i in range(len(y_true)):
        index = np.where(y_pred[i] == y_true[i])[0]
        if len(index) == 0:
            continue
        ngdcg += get_dcg(index[0])
    return ngdcg / len(y_true)

# Xử lý tuổi với trung bình tuổi

In [8]:
transform_x_train = torch.tensor(data_train_avg.drop(["country_destination"], axis = 1).values, dtype=torch.float)
transform_y_train = torch.tensor(data_train_avg["country_destination"].values, dtype=torch.long)

transform_x_valid = torch.tensor(data_valid_avg.drop(["country_destination"], axis = 1).values, dtype = torch.float)
transform_y_valid = torch.tensor(data_valid_avg["country_destination"].values, dtype = torch.long)

transform_x_test = torch.tensor(data_test_avg.drop(["id"], axis = 1).values, dtype = torch.float)

In [9]:
epochs = 100
history_accuracy = [0]
t = 0

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
batch_size = 64

while True:
    print(f"Epochs {t+1}\n.....................................")
    t+=1
    train(transform_x_train, transform_y_train, model, loss_fn, optimizer)
    pred = model(transform_x_valid)
    pred = torch.argsort(pred, dim = 1, descending = True)[:, :5]
    ndcg = get_ndcg(pred, transform_y_valid)
    history_accuracy.append(ndcg)
    print(f"Accuracy on validation set: {ndcg.item():>7f}")
    
    if(history_accuracy[-1] - history_accuracy[-2] < 0.00001 or t > epochs):
        break
print("Done!")

Epochs 1
.....................................
loss: 2.682527, [ current: 64/170760 ]
loss: 1.290052, [ current: 12864/170760 ]
loss: 1.099663, [ current: 25664/170760 ]
loss: 1.091501, [ current: 38464/170760 ]
loss: 1.117885, [ current: 51264/170760 ]
loss: 1.025038, [ current: 64064/170760 ]
loss: 1.256730, [ current: 76864/170760 ]
loss: 0.961873, [ current: 89664/170760 ]
loss: 1.113633, [ current: 102464/170760 ]
loss: 0.924520, [ current: 115264/170760 ]
loss: 1.367191, [ current: 128064/170760 ]
loss: 1.203486, [ current: 140864/170760 ]
loss: 1.206839, [ current: 153664/170760 ]
loss: 1.057565, [ current: 166464/170760 ]
Accuracy on validation set: 0.809354
Epochs 2
.....................................
loss: 1.029370, [ current: 64/170760 ]
loss: 1.189406, [ current: 12864/170760 ]
loss: 1.075436, [ current: 25664/170760 ]
loss: 1.078231, [ current: 38464/170760 ]
loss: 1.104217, [ current: 51264/170760 ]
loss: 1.014079, [ current: 64064/170760 ]
loss: 1.243939, [ current: 76

In [10]:
for ind, item in enumerate(history_accuracy):
    print(f"Epoch {ind}:\n \t Accuracy: {item}")

Epoch 0:
 	 Accuracy: 0
Epoch 1:
 	 Accuracy: 0.8093543175441609
Epoch 2:
 	 Accuracy: 0.8099500250422496
Epoch 3:
 	 Accuracy: 0.810801608944883
Epoch 4:
 	 Accuracy: 0.810691425533897


In [11]:
id_user = np.array(data_test_avg["id"])

pred_avg = model(transform_x_test)
pred_avg = np.array(torch.argsort(pred_avg, dim = 1, descending = True)[:, :5])

ids = []
country = []

dict_ = {'AU': 10,
 'CA': 3,
 'DE': 2,
 'ES': 11,
 'FR': 5,
 'GB': 1,
 'IT': 9,
 'NDF': 4,
 'NL': 7,
 'PT': 6,
 'US': 0,
 'other': 8}

inverse_dict = {v: k for k,v in dict_.items()}

for i in range(len(id_user)):
    ids += [id_user[i]]*5
    for j in range(5):
        country.append(inverse_dict[pred_avg[i, j]])

In [12]:
result_avg = {"id": ids, "country": country}

submission_avg = pd.DataFrame(result_avg)

submission_avg.to_csv("submission_avg.csv", index=False)

# Xử lý tuổi với kỹ thuật KNN

In [13]:
transform_x_train_knn = torch.tensor(data_train_knn.drop(["country_destination"], axis = 1).values, dtype=torch.float)
transform_y_train_knn = torch.tensor(data_train_knn["country_destination"].values, dtype=torch.long)

transform_x_valid_knn = torch.tensor(data_valid_knn.drop(["country_destination"], axis = 1).values, dtype = torch.float)
transform_y_valid_knn = torch.tensor(data_valid_knn["country_destination"].values, dtype = torch.long)

transform_x_test_knn = torch.tensor(data_test_knn.drop(["id"], axis = 1).values, dtype = torch.float)

In [14]:
epochs = 100
hist_knn = [0.0]
t = 0

model_knn = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_knn.parameters(), lr = 1e-3)
batch_size = 64

while True:
    print(f"Epochs {t+1}\n.....................................")
    t+=1
    train(transform_x_train_knn, transform_y_train_knn, model_knn, loss_fn, optimizer)
    pred = model(transform_x_valid_knn)
    pred = torch.argsort(pred, dim = 1, descending = True)[:, :5]
    ndcg = get_ndcg(pred, transform_y_valid_knn)
    hist_knn.append(ndcg)
    print(f"Accuracy on validation set: {ndcg.item():>7f}")
    
    if(hist_knn[-1] - hist_knn[-2] < 0.00001 or t > epochs):
        break
print("Done!")

Epochs 1
.....................................
loss: 2.260841, [ current: 64/170760 ]
loss: 1.527574, [ current: 12864/170760 ]
loss: 1.291462, [ current: 25664/170760 ]
loss: 0.982104, [ current: 38464/170760 ]
loss: 1.075217, [ current: 51264/170760 ]
loss: 0.963481, [ current: 64064/170760 ]
loss: 1.037112, [ current: 76864/170760 ]
loss: 0.912206, [ current: 89664/170760 ]
loss: 1.228952, [ current: 102464/170760 ]
loss: 0.950976, [ current: 115264/170760 ]
loss: 1.297207, [ current: 128064/170760 ]
loss: 1.079788, [ current: 140864/170760 ]
loss: 1.042054, [ current: 153664/170760 ]
loss: 1.162449, [ current: 166464/170760 ]
Accuracy on validation set: 0.702757
Epochs 2
.....................................
loss: 1.157128, [ current: 64/170760 ]
loss: 1.403185, [ current: 12864/170760 ]
loss: 1.208939, [ current: 25664/170760 ]
loss: 0.963823, [ current: 38464/170760 ]
loss: 1.042769, [ current: 51264/170760 ]
loss: 0.931745, [ current: 64064/170760 ]
loss: 1.024565, [ current: 76

In [15]:
id_user = np.array(data_test_knn["id"])

pred_knn = model_knn(transform_x_test_knn)
pred_knn = np.array(torch.argsort(pred_knn, dim = 1, descending = True)[:, :5])

ids = []
country = []

dict_ = {'AU': 10,
 'CA': 3,
 'DE': 2,
 'ES': 11,
 'FR': 5,
 'GB': 1,
 'IT': 9,
 'NDF': 4,
 'NL': 7,
 'PT': 6,
 'US': 0,
 'other': 8}

inverse_dict = {v: k for k,v in dict_.items()}

for i in range(len(id_user)):
    ids += [id_user[i]]*5
    for j in range(5):
        country.append(inverse_dict[pred_knn[i, j]])

In [16]:
result_knn = {"id": ids, "country": country}

submission_knn = pd.DataFrame(result_avg)

submission_knn.to_csv("submission_knn.csv", index=False)